In [1]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD! https://stackoverflow.com/questions/31984196/ipython-notebook-run-all-cells-on-open
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
               //IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
           // $('#header-container').hide();
          //$('#maintoolbar').hide();   
        }
    );

</script>

In [2]:
from IPython.display import HTML, clear_output,Javascript, display
from ipywidgets import widgets
from ipywidgets import Button, GridBox, Layout, ButtonStyle, Box, VBox
import requests
import json
import pandas as pd
from pprintpp import pprint as pp
from pandas.io.json import json_normalize
import time
import datetime as dt
import pytz
import base64
import qgrid

#pd.set_option('display.max_rows', 999)
# I wish i knew how to always hide this cell !

# button[data-event-type$=_row] {
#    display: none !important;}

# button.btn.btn-default.fa.fa-arrows-alt.full-screen-btn { margin-left:5px;}


HTML('''
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>
<style>
button[data-event-type$=_row] {
    display: none !important;}

 button.btn.btn-default.fa.fa-arrows-alt.full-screen-btn { margin-left:5px;}
</style>''')


# Welcome 

Here I will try to make a dashboard to parallelize running reports.  
What I need to do is get better at setting up docker images and parallelizing with ipyparallel



#### API documentation references 
 - [CANVAS API DOC](https://canvas.instructure.com/doc/api/index.html)
 - [Requests Python Library](http://docs.python-requests.org/en/master/)

<hr>


In [3]:
error = widgets.Output()

API_widget = widgets.Text(placeholder='API TOKEN',
                       layout=Layout(width='90%',align_items='center', grid_area='main'))


Authentication = Button(
    value=False,
    description='NOT Authorized',
    disabled=True,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    icon='unlock',
    layout=Layout(width='auto', grid_area='header')
)


Submit = Button(
    value=False,
    description='Submit',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    icon='',
    layout=Layout(width='auto', grid_area='footer')
)



# for the env // Domain
environ = widgets.Select(
    options=['Production', 'Test', 'Beta'],
    value='Production',
    description='Environment:',
    disabled=False,
    rows=3,
    layout=Layout(width='auto', grid_area='sidebar')
)

environs = {'Production':'https://canvas.upenn.edu/api/v1/', 'Test':'b', 'Beta':'c'}



def set_up_forms():
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+11)'))

def authorize():
    # check api key 
    global headers
    headers = {
        'Authorization': 'Bearer %s' % (API_widget.value) ,
    }
    global domain
    domain = environs[environ.value]
    
    x = test_api() # returns status code of test response
    
    if API_widget.value and x ==200:
    # if good:
        
        # UI changes
        Authentication.icon = 'lock'
        Authentication.description = 'Authorized'
        Authentication.button_style ='success'
        set_up_forms()
    #Authentication.style.button_color = 'lightgreen'
    # if bad:
    else:
        error.append_stdout("Invalid API token")
        Authentication.description='NOT Authorized'
        Authentication.button_style='warning' # 'success', 'info', 'warning', 'danger' or ''
        Authentication.icon='unlock'
    # output note 
    


def on_button_clicked(b):
    # clear any output ( error output from authorize )
    error.clear_output()
    authorize()
    
    
Submit.on_click(on_button_clicked)


#https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html#The-Grid-layout

header  = Authentication
main    = API_widget
sidebar = environ
footer  = Submit



form = GridBox(children=[header, main, sidebar,footer],
        layout=Layout(
            width='80%',
            margin="2% 0% 2% 10%",
            grid_template_rows='auto auto auto',
            grid_template_columns='25% 25% 25% 25%',
            #grid_gap='5px 5px',
            grid_template_areas='''
            " . header header ."
            "main main sidebar sidebar"
            " . footer footer . "
            ''')
       )

display(HTML("<h2 style='text-align: center;'>Please enter your API Credentials</h2>"))
display(form)
display(error)



def on_value_change(change):
    Authentication.description='NOT Authorized'
    Authentication.button_style='warning' # 'success', 'info', 'warning', 'danger' or ''
    Authentication.icon='unlock'
    headers = {
    'Authorization': 'Bearer %s' % (change['new']) ,
    }
    

API_widget.observe(on_value_change, names='value')



environs = {'Production':'https://canvas.upenn.edu/api/v1/', 'Test':'b', 'Beta':'c'}
domain = environs[environ.value]


def test_api():
    
    #headers = {
    #'Authorization': 'Bearer %s' % (API_widget.value) ,
    #}
    
    url = domain + 'accounts/self/scopes'
    response = requests.get(url,headers=headers)
    r = response.json()
    #print(response.status_code)
    return response.status_code



######################################
######### HELPER FUNCTION ############
######################################

# utc to est convert and back 
# all user input is in est in the basic time format
# example: 2019-05-14 1am
# output from canvas 2019-05-14T14:28:08Z (utc)

def est_to_utc(datestring):
    utc=pytz.utc
    eastern=pytz.timezone('US/Eastern')
    fmt='%Y-%m-%d %H:%M'
    date=dt.datetime.strptime(datestring,"%Y-%m-%d %I%p")
    date_eastern=eastern.localize(date,is_dst=None)
    date_utc=date_eastern.astimezone(utc)
    return(date_utc.strftime(fmt))
    
def user_info(user_id):
    #url:GET|/api/v1/users/:id
    url = domain + 'users/'+user_id
    response = requests.get(url,headers=headers)
    return(response.json())


def create_download_link( df, filename):  
    title = "Download CSV file"
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

GridBox(children=(Button(button_style='warning', description='NOT Authorized', disabled=True, icon='unlock', l…

Output()

<hr>

## Table of Contents

1. [LTI Usage](#Explore-LTI-Usage-🔎)
1. [Analytics](#Explore-Analytics-📊)
2. 

<hr>

## Explore LTI Usage 🔎

#### Description
... coming soon ...
##### A Tool is Only as Good as its User ...
please have a good idea what you are looking for before you start running reports

In [4]:
# LTI report



<hr>

## Explore Analytics 📊

#### Description
... coming soon ...  
[CANVAS DOC](https://canvas.instructure.com/doc/api/analytics.html)
##### A Tool is Only as Good as its User ...
please have a good idea what you are looking for before you start running reports

In [5]:
# https://canvas.instructure.com/doc/api/analytics.html

<hr>

###### To do 
- 😪 Finish first report!
 

###### Sources
- https://ipyparallel.readthedocs.io/en/latest/demos.html
- https://buildmedia.readthedocs.org/media/pdf/jupyter/latest/jupyter.pdf
